In [ ]:

bucket_name = 'hawaii-marine-debris'
file_path_template = "{}/jpg/{}"
downloaded_tile = set()
csv_rows = []
islands= ['niihau']#'lanai','bigisland',''

local_tmp_folder = "/home/paperspace/data"
# /home/paperspace/data/images/niihau/
local_tmp_image_folder = "/home/paperspace/data/niihau/images/{filename}"
tile_with_debris = []


In [ ]:
# mkdir /tmp/niihau/

In [ ]:
import os
import boto3 
import boto3 
boto3.setup_default_session(profile_name='hawaii')
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
version='v2'

s3_resource.Bucket(bucket_name).download_file("niihau/all_tiles.csv", "{}/all_tiles.csv".format(local_tmp_folder))
# s3://hawaii-marine-debris/ml/model/debris_090818.h5
def download_image_from_s3(s3_key, island='niihau'):
    basename = os.path.basename(s3_key)
    local_file_path = local_tmp_image_folder.format(island=island, filename=basename)
    s3_resource.Bucket(bucket_name).download_file(s3_key, local_file_path)
    return local_file_path

In [ ]:
s3_resource.Bucket(bucket_name).download_file("ml/model/debris_090818.h5", "{}/debris_090818.h5".format(local_tmp_folder))


In [ ]:
# s3_resource.Bucket(bucket_name).download_file("ml//all_tiles.csv", "{}/all_tiles.csv".format(local_tmp_folder))
s3_resource.Bucket(bucket_name).download_file("ml/v2/data/labels.csv", "{}/labels.csv".format(local_tmp_folder))

# s3://hawaii-marine-debris/

In [ ]:
"{}/debris_090818.h5".format(local_tmp_folder)

In [ ]:
import pandas as pd
df  =pd.read_csv("{}/all_tiles.csv".format(local_tmp_folder))

df['cleaner_label'] = df['label'].replace({'B':'Buoys','C': 'Cloth','F':'Foam','L':'Line','M':'Metal','N':'Net','P':'Plastic','T':'Tire','W':'Processedwood','V':'Boat','O':'Other'})
# Bicycle,0
# Boat,1
# Surfboard,2
# Bottle,3
# Plastic bag,4
# Wheel,5
# Tire,6
# Buoys,7
# Cloth,8
# Foam,9
# Line,10
# Metal,11
# Net,12
# Plastic,13
# Processedwood,14
# Other,15

# Debris categories:
# B = Buoys and floats
# C = Cloth 
# F = Foam 
# L = Line (single pieces of rope, not net)
# M = Metal
# N = Net
# P = Plastic
# T = Tire
# W = Processed wood
# V = Vessel
# O = Other 

len(df[df['cleaner_label'].isna()])

In [ ]:
df.groupby('label').count()

In [ ]:
df['x0']= df['x0'].fillna(0).astype(int)
df['x1']= df['x1'].fillna(0).astype(int)
df['y0']= df['y0'].fillna(0).astype(int)
df['y1']= df['y1'].fillna(0).astype(int)
# df['x0'] = df['x0'].fillna(0).astype(int)
df

In [ ]:
local_tmp_image_folder = "/home/paperspace/data/images/niihau/{filename}"

def local_file_path(s3_key, island='niihau'):
    basename = os.path.basename(s3_key)
    local_file_path = local_tmp_image_folder.format(island=island, filename=basename)
    return local_file_path
df['local_file_path'] = df['s3_key'].apply(local_file_path)

In [ ]:
import numpy as np
msk = np.random.rand(len(df)) < 0.9

train = df[msk]
test = df[~msk]

train[['local_file_path','x0','y1','x1','y0','cleaner_label']].to_csv('keras_annotations_train.csv', encoding='utf-8', header=False, index=False)
test[['local_file_path','x0','y1','x1','y0','cleaner_label']].to_csv('keras_annotations_test.csv', encoding='utf-8', header=False, index=False)


In [ ]:
s3_client.upload_file('keras_annotations_train.csv', bucket_name, 'ml/v3/data/keras_annotations_train.csv')
s3_client.upload_file('keras_annotations_test.csv', bucket_name, 'ml/v3/data/keras_annotations_test.csv')

In [ ]:
test[['local_file_path','x0','y1','x1','y0','cleaner_label']]

In [ ]:

# for index, row in df[:20].iterrows():
#     local_file = download_image_from_s3(row['s3_key'])
#     print(local_file)